In [ ]:
# JobsDB

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotVisibleException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By

# Following arguments would differ for different Users
username = "Username"
password = "Password"
webdriverPath = 'C:\Users\Tushar\Downloads\chromedriver_win32\chromedriver.exe'
url = "https://hk.jobsdb.com/hk"
searchID = "JobAlertListRunSearchResult_Link_1867257_HK"
lastPageNumber = 20 # If you don't know you can just set a high value as script will throw an error when that page does not exist

driver = webdriver.Chrome(executable_path=webdriverPath)
driver.get(url)

# Login to the User Account
driver.find_element_by_class_name("fn-header-canlogin").click()
driver.find_element_by_id("c_JbSrP1LnItDap_El0").send_keys(username)
driver.find_element_by_id("c_JbSrP1LnItDap_Pd0").send_keys(password)
driver.find_element_by_id("reg-login-button").click()

# Run already created search in the account
element_present = EC.presence_of_element_located((By.CLASS_NAME, "profile-owner-container"))
WebDriverWait(driver, 3).until(element_present)
driver.find_element_by_class_name("profile-owner-container").click()
driver.find_element_by_class_name("dropdown-menu-myjobsdb").click()
driver.find_element_by_id(searchID).click()

pageNumber = 0

#Apply for jobs page by page
def apply():
    jobs = 'job-title'

    try:
        element_present = EC.presence_of_element_located((By.CLASS_NAME, jobs))
        WebDriverWait(driver, 3).until(element_present)
        a = driver.find_elements_by_class_name(jobs)

        i = 0
        while i < len(a) :
            
            # Try-Catch to handle pop-ups
            try:
                driver.execute_script("arguments[0].scrollIntoView();", a[i])
                a[i].click()
            except WebDriverException:
                driver.find_element_by_class_name("layer-cross").click()
                continue
            driver.execute_script("arguments[0].scrollIntoView();", driver.find_element_by_id("Btn_Apply"))
            driver.find_element_by_id("Btn_Apply").click()
                
            driver.switch_to.window(driver.window_handles[1])
            
            # Try-Catch for already applied jobs or jobs with external application
            try :
                driver.find_element_by_xpath("//button[@name='BtnProfileApply']").click()
            except NoSuchElementException:
                print "Already Applied for : ", i
                 
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
            a = driver.find_elements_by_class_name(jobs)
            i += 1

    except TimeoutException:
        print "Timed out waiting for page to load"

while pageNumber < lastPageNumber :
    apply()
    print "Page Done : ", pageNumber
    pageNumber += 1
    driver.execute_script("searchResultPagination.MoveToPage("+str(pageNumber)+");")
    
print "Finished"
driver.close()